In [1]:
from concurrentbuffer.commander import Commander
class IndexCommander(Commander):
    def __init__(self, max_index: int):
        self._max_index = max_index
        self._index = 0

    def create_message(self) -> dict:
        message = {"index": self._index}
        self._index = (self._index + 1) % self._max_index
        return message

In [2]:
import numpy as np
from concurrentbuffer.producer import Producer

class DataProducer(Producer):
    def __init__(self, data_shape: tuple):
        self._data_shape = data_shape

    def create_data(self, message: dict) -> np.ndarray:
        index = message['index']
        return self._time_consuming_processing(index)

    def _time_consuming_processing(self, index) -> np.ndarray:
        return np.ones(self._data_shape)*index
        

In [21]:
from concurrentbuffer.iterator import buffer_iterator_factory


# the number of cpus/producers
cpus = 4

# the buffershape in the shared memory
buffer_shape = (1, 10)

# the context of multiprocess (spawn or fork)
context = 'fork'

# if the messages from the commander and the produced data are first in first out
deterministic = True

# You will have to create your own Commander class, please see instructions below
# a user defined commander, should subclass the Commander class
commander = IndexCommander(max_index=10)

# You will have to create your own Producer class, please see instructions below
# a user defined producer, should subclass the Producer class
producer = DataProducer(data_shape=buffer_shape)

# create a buffer iterator
buffer_iterator = buffer_iterator_factory(
    cpus=cpus,
    buffer_shape=buffer_shape,
    commander=commander,
    producer=producer,
    context=context,
    deterministic=deterministic,
)

# loop through the buffer that is filled concurrently at the same time
for index in range(10):
    data = next(buffer_iterator)
    print(data)
    
# always stop the iterator to close all processes and free the shared memory
buffer_iterator.stop()


[[0 0 0 0 0 0 0 0 0 0]]
[[1 1 1 1 1 1 1 1 1 1]]
[[2 2 2 2 2 2 2 2 2 2]]
[[3 3 3 3 3 3 3 3 3 3]]
[[4 4 4 4 4 4 4 4 4 4]]
[[5 5 5 5 5 5 5 5 5 5]]
[[6 6 6 6 6 6 6 6 6 6]]
[[7 7 7 7 7 7 7 7 7 7]]
[[8 8 8 8 8 8 8 8 8 8]]
[[9 9 9 9 9 9 9 9 9 9]]
